# Introduction to Python and Natural Language Technologies

# Lecture 5 

## Decorators and packaging

### March 7, 2018

## Let's create a greeter function

- takes another function as a parameter
- greets the caller before calling the function

In [1]:
def greeter(func):
    print("Hello")
    func()
    
def say_something():
    print("Let's learn some Python.")
    
greeter(say_something)
# greeter(12)

Hello
Let's learn some Python.


## Functions are first class objects

- they can be passed as arguments
- they can be returned from other functions (example later)

## Let's create a `count_predicate` function

- takes a iterable and a predicate (yes-no function)
- calls the predicate on each element
- counts how many times it returns True
- same as `std::count` in C++

In [2]:
def count_predicate(predicate, iterable):
    true_count = 0
    for element in iterable:
        if predicate(element) is True:
            true_count += 1
    return true_count

## Q. Can you write this function in fewer lines?

In [3]:
def count_predicate(predicate, iterable):
    return sum(predicate(e) for e in iterable)

### The predicate parameter

- it can be anything 'callable'

#### 1. function

In [4]:
def is_even(number):
    return number % 2 == 0

numbers = [1, 3, 2, -5, 0, 0]

count_predicate(is_even, numbers)

3

#### 2. instance of a class that implements `__call__` (functor)

In [5]:
class IsEven(object):
    def __call__(self, number):
        return number % 2 == 0
    
print(count_predicate(IsEven(), numbers))

IsEven()(123)
i = IsEven()
i(11)

3


False

#### 3. lambda expression

In [6]:
count_predicate(lambda x: x % 2 == 0, numbers)

3

## Functions can be nested

In [7]:
def parent():
    print("I'm the parent function")
    
    def child():
        print("I'm the child function")
        
parent()

I'm the parent function


the nested function is only accessible from the parent

In [8]:
def parent():
    print("I'm the parent function")
    
    def child():
        print("I'm the child function")
    
    print("Calling the nested function")
    child()
        
parent()
# parent.child  # raises AttributeError

I'm the parent function
Calling the nested function
I'm the child function


## Functions can be return values

In [9]:
def parent():
    print("I'm the parent function")
    
    def child():
        print("I'm the child function")
        
    return child

child_func = parent()

print("Calling child")
child_func()


print("\nUsing parent's return value right away")
parent()()

I'm the parent function
Calling child
I'm the child function

Using parent's return value right away
I'm the parent function
I'm the child function


## Nested functions have access to the parent's scope

closure

In [10]:
def parent(value):
    
    def child():
        print("I'm the nested function. "
              "The parent's value is {}".format(value))
        
    return child
        
child_func = parent(42)

print("Calling child_func")
child_func()

Calling child_func
I'm the nested function. The parent's value is 42


In [11]:
f1 = parent("abc")
f2 = parent(123)

f1()
f2()

f1 is f2

I'm the nested function. The parent's value is abc
I'm the nested function. The parent's value is 123


False

## Function factory

In [12]:
def make_func(param):
    value = param
    
    def func():
        print("I'm the nested function. "
              "The parent's value is {}".format(value))
        
    return func

func_11 = make_func(11)
func_abc = make_func("abc")

func_11()
func_abc()

I'm the nested function. The parent's value is 11
I'm the nested function. The parent's value is abc


## Wrapper function factory

- let's create a function that takes a function return an almost identical function
- the returned function adds some logging

In [13]:
def add_noise(func):
    
    def wrapped_with_noise():
        print("Calling function {}".format(func.__name__))
        func()
        print("{} finished.".format(func.__name__))
        
    return wrapped_with_noise

### Wrapping a function

The function we are going to wrap:

In [14]:
def noiseless_function():
    print("This is not noise")
    
noiseless_function()

This is not noise


#### now add some noise

In [15]:
noisy_function = add_noise(noiseless_function)

noisy_function()

Calling function noiseless_function
This is not noise
noiseless_function finished.


#### Bound the original reference to the wrapped function

- i.e. `greeter` should refer to the wrapped function
- we don't need the original function

In [16]:
def greeter():
    print("Hello")
    
print(id(greeter))
   
greeter = add_noise(greeter)
greeter()
print(id(greeter))

140476910623328
Calling function greeter
Hello
greeter finished.
140476910760552


#### this turns out to be a frequent operation

In [17]:
def friendly_greeter():
    print("Hello friend")
    
def rude_greeter():
    print("Hey you")
    
friendly_greeter = add_noise(friendly_greeter)
rude_greeter = add_noise(rude_greeter)
friendly_greeter()

rude_greeter()

Calling function friendly_greeter
Hello friend
friendly_greeter finished.
Calling function rude_greeter
Hey you
rude_greeter finished.


## Decorator syntax

- a decorator is a function
  - that takes a function as an argument
  - returns a wrapped version of the function
- the decorator syntax is just syntactic sugar (shorthand) for:

```python
func = decorator(func)
```

In [18]:
@add_noise
def informal_greeter():
    print("Yo")
    
# informal_greeter = add_noise(informal_greeter)
    
informal_greeter()

Calling function informal_greeter
Yo
informal_greeter finished.


### Pie syntax

- introduced in [PEP318](https://www.python.org/dev/peps/pep-0318/) in Python 2.4
- various syntax proposals were suggested, summarized [here](https://wiki.python.org/moin/PythonDecorators#A1._pie_decorator_syntax)

# Problem 1. Function metadata is lost

In [19]:
informal_greeter.__name__

'wrapped_with_noise'

### Solution 1. Copy manually

In [20]:
def add_noise(func):
    
    def wrapped_with_noise():
        print("Calling {}...".format(func.__name__))
        func()
        print("{} finished.".format(func.__name__))
        
    wrapped_with_noise.__name__ = func.__name__
    return wrapped_with_noise

@add_noise
def greeter():
    """meaningful documentation"""
    print("Hello")
    
print(greeter.__name__)

greeter


What about other metadata such as the docstring?

In [21]:
print(greeter.__doc__)

None


### Solution 2. `functools.wraps`

In [22]:
from functools import wraps

def add_noise(func):
    
    @wraps(func)
    def wrapped_with_noise():
        print("Calling {}...".format(func.__name__))
        func()
        print("{} finished.".format(func.__name__))
        
    wrapped_with_noise.__name__ = func.__name__
    return wrapped_with_noise

@add_noise
def greeter():
    """function that says hello"""
    print("Hello")
    
print(greeter.__name__)
print(greeter.__doc__)

greeter
function that says hello


## Problem 2. Function arguments

- so far we have only decorated functions without parameters
- to wrap arbitrary functions, we need to capture a variable number of arguments
- remember `args` and `kwargs`

In [23]:
def function_with_variable_arguments(*args, **kwargs):
    print(args)
    print(kwargs)
    
function_with_variable_arguments(1, "apple", tree="peach")

(1, 'apple')
{'tree': 'peach'}


#### the same mechanism can be used in decorators

In [24]:
def add_noise(func):
    
    @wraps(func)
    def wrapped_with_noise(*args, **kwargs):
        print("Calling {}...".format(func.__name__))
        func(*args, **kwargs)
        print("{} finished.".format(func.__name__))
        
    return wrapped_with_noise

- the decorator has only one parameter: `func`, the function to wrap
- the returned function (`wrapped_with_noise`) takes arbitrary parameters: `args`, `kwargs`
- it calls `func`, the decorator's argument with arbitrary parameters

In [25]:
@add_noise
def personal_greeter(name):
    print("Hello {}".format(name))
    
personal_greeter("John")

Calling personal_greeter...
Hello John
personal_greeter finished.


# Decorators can take parameters too

- they have to return a decorator without parameters - decorator factory

In [26]:
def decorator_with_param(param1, param2=None):
    print("Creating a new decorator: {0}, {1}".format(
        param1, param2))
    def actual_decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print("Wrapper function {}".format(
                func.__name__))
            print("Params: {0}, {1}".format(param1, param2))
            return func(*args, **kwargs)
        return wrapper
    return actual_decorator

In [27]:
@decorator_with_param(42, "abc")
def personal_greeter(name):
    print("Hello {}".format(name))
    
@decorator_with_param(4)
def personal_greeter2(name):
    print("Hello {}".format(name))
    
print("\nCalling personal_greeter")
personal_greeter("Mary")

Creating a new decorator: 42, abc
Creating a new decorator: 4, None

Calling personal_greeter
Wrapper function personal_greeter
Params: 42, abc
Hello Mary


In [28]:
def hello(name):
    print("Hello {}".format(name))
    
hello = decorator_with_param(1, 2)(hello)
hello("john")

Creating a new decorator: 1, 2
Wrapper function hello
Params: 1, 2
Hello john


# Decorators can be implemented as classes

- `__call__` implements the wrapped function

In [29]:
class MyDecorator(object):
    def __init__(self, func):
        self.func_to_wrap = func
        wraps(func)(self)
        
    def __call__(self, *args, **kwargs):
        print("before func {}".format(self.func_to_wrap.__name__))
        res = self.func_to_wrap(*args, **kwargs)
        print("after func {}".format(self.func_to_wrap.__name__))
        return res
    
@MyDecorator
def foo():
    print("bar")

foo()

before func foo
bar
after func foo


# See also

Decorator overview with some advanced techniques: https://www.youtube.com/watch?v=9oyr0mocZTg

A very deep dive into decorators: https://www.youtube.com/watch?v=7jGtDGxgwEY

# Functional Python: map, filter and reduce

Python has 3 built-in functions that originate from functional programming.

## Map

- `map` applies a function on each element of a sequence

In [30]:
def double(e):
    return e * 2

l = [2, 3, "abc"]

list(map(double, l))

[4, 6, 'abcabc']

In [31]:
map(double, l)

In [32]:
list(map(lambda x: x * 2, [2, 3, "abc"]))

[4, 6, 'abcabc']

## Filter

- filter creates a list of elements for which a function returns true

In [33]:
def is_even(n):
    return n % 2 == 0

l = [2, 3, -1, 0, 2]

list(filter(is_even, l))

[2, 0, 2]

In [34]:
list(filter(lambda x: x % 2 == 0, range(8)))

[0, 2, 4, 6]

### Most comprehensions can be rewritten using map and filter

In [35]:
l = [2, 3, 0, -1, 2, 0, 1]

signum = [x / abs(x) if x != 0 else x for x in l]
print(signum)

[1.0, 1.0, 0, -1.0, 1.0, 0, 1.0]


In [36]:
list(map(lambda x: x / abs(x) if x != 0 else 0, l))

[1.0, 1.0, 0, -1.0, 1.0, 0, 1.0]

In [37]:
even = [x for x in l if x % 2 == 0]
print(even)

[2, 0, 2, 0]


In [38]:
print(list(filter(lambda x: x % 2 == 0, l)))

[2, 0, 2, 0]


## Reduce

- reduce applies a rolling computation on a sequence
- the first argument of `reduce` is two-argument function
- the second argument is the sequence
- the result is accumulated in an accumulator

In [39]:
from functools import reduce

l = [1, 2, -1, 4]
reduce(lambda x, y: x*y, l)

-8

an initial value for the accumulator may be supplied

In [40]:
reduce(lambda x, y: x*y, l, 10)

-80

In [41]:
reduce(lambda x, y: max(x, y), l)
reduce(max, l)

4

In [42]:
reduce(lambda x, y: x + int(y % 2 == 0) * y, l, 0)

6

# Modules and imports

- `import` statement combines two operations
    1. it searches for the named module, 
    2. then it binds the results of that search to a name in the local scope -- [official documentation](https://docs.python.org/3/reference/import.html) (emphasis mine)
- several formats

### importing full modules

In [43]:
import sys

", ".join(dir(sys))

'__displayhook__, __doc__, __excepthook__, __loader__, __name__, __package__, __spec__, __stderr__, __stdin__, __stdout__, _clear_type_cache, _current_frames, _debugmallocstats, _getframe, _git, _xoptions, abiflags, api_version, argv, base_exec_prefix, base_prefix, builtin_module_names, byteorder, call_tracing, callstats, copyright, displayhook, dont_write_bytecode, exc_info, excepthook, exec_prefix, executable, exit, flags, float_info, float_repr_style, get_asyncgen_hooks, get_coroutine_wrapper, getallocatedblocks, getcheckinterval, getdefaultencoding, getdlopenflags, getfilesystemencodeerrors, getfilesystemencoding, getprofile, getrecursionlimit, getrefcount, getsizeof, getswitchinterval, gettrace, hash_info, hexversion, implementation, int_info, intern, is_finalizing, maxsize, maxunicode, meta_path, modules, path, path_hooks, path_importer_cache, platform, prefix, ps1, ps2, ps3, real_prefix, set_asyncgen_hooks, set_coroutine_wrapper, setcheckinterval, setdlopenflags, setprofile, set

### importing submodules

In [44]:
from os import path

try:
    os
except NameError:
    print("os does not seem to be defined")
    
try:
    path
    print("path found")
except NameError:
    print("path does not seem to be defined")

os does not seem to be defined
path found


the `as` keyword binds the module to a different name:

In [45]:
import os as os_module

try:
    os
except NameError:
    print("os does not seem to be defined")
    
try:
    os_module
    print("os_module found")
except NameError:
    print("os_module does not seem to be defined")

os does not seem to be defined
os_module found


### importing more than one module/submodule

In [46]:
# import os, sys
from sys import stdin, stderr, stdout

### importing functions or classes

In [47]:
from argparse import ArgumentParser
import inspect

inspect.isclass(ArgumentParser)

True

### importing everything from a module

**NOT** recommended because we have no way of knowing where names come from

In [48]:
from os import *

try:
    makedirs
    stat
    print("everything found")
except NameError:
    print("Something not found")

everything found


# Packaging

Python projects can be packaged and distributed.

## Naming convention

- all lowercase
- underscore separated, no hyphens
- unique on PyPI

## Minimal structure

~~~
example_package/
    example_package/
        __init__.py
    setup.py
~~~

- the source code is located in a separate subdirectory with the same name
  - just a convention, not mandatory
- `setup.py` describes how the package should be installed

## Source code

- each directory that has a `__init__.py` file is going to be a subpackage
  - `__init__.py` may be empty
  
## setup.py

Demo

## Nice to have

- licence
- `Manifest.IN` - list of additional files
- `setup.cfg` - option defaults for `setup.py`
- `README.rst` - `README` using reStructuredText

https://github.com/pypa/sampleproject

## See also

https://packaging.python.org/tutorials/distributing-packages/

# Pip, virtualenv, Anaconda

1. Pip
  - package installer
2. Virtualenv
  - Python environment manager
  - a virtualenv is a Python environment separate from the system Python install
  - advantages
    - different Python version than the system default may be used
    - different package versions may be used
    - updates and package installs do not affect the system install
    - no need for root/Admin access
  - activate and deactivate
  - virtualenvwrapper is a collection of helper scripts (mainly for Linux)
3. Anaconda
  - package installer and environment manager
  - scientific packages included

# Global Interpreter Lock (GIL)

- CPython, the reference implementation has a reference counting garbage collector
- reference counting GC is **not** thread-safe :(
- "GIL, is a mutex that protects access to Python objects, preventing multiple threads from executing Python bytecodes at once"
- IO, image processing and Numpy (numerical computation and matrix library) heavy lifting happens outside the GIL
- other computations cannot fully take advantage of multithreading :(
- Jython and IronPython do not have a GIL

## See also

[Python wiki page on the GIL](https://wiki.python.org/moin/GlobalInterpreterLock)

[Live GIL removal (advanced)](https://www.youtube.com/watch?v=pLqv11ScGsQ)